In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
#Read the data into dataframe
car_df = pd.read_csv('newCardata.csv')
car_features = pd.read_csv('finalDataPreprocessBinary.csv')
car_label = car_df['FraudFound']



In [3]:
#change the label of the data
labelNo = LabelEncoder()
car_df['FraudFound'] = labelNo.fit_transform(car_df['FraudFound'].astype('str'))
car_label = car_df['FraudFound']
print(car_label.shape)


(15419,)


In [4]:
#split the data into train and test
X_train,X_test,y_train,y_test = train_test_split(car_features,car_label,random_state=3,test_size=0.25)
print('xtrain:',type(X_train))

xtrain: <class 'pandas.core.frame.DataFrame'>


In [5]:
#model object
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)
print('Random forest classifier:')
predicted = model.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,model.predict(X_test)) * 100,2))


Random forest classifier:
<class 'numpy.ndarray'> (3855,) [0 0 0 ..., 0 0 0]
Accuracy is  93.72


In [6]:
# calculating specifity and sensitivity
# 0  := Negative
# 1 := Positive
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()
print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

Confusion Matrix:
 [[3606    3]
 [ 239    7]]
TN: 3606
FP: 3
FN: 239
TP: 7
Accuracy: 93.7224383917
Sensitivity: 2.84552845528
Specificity: 99.9168744805


In [7]:

#converting pandas.core.series.Series to numpy.ndarray
print(type(y_test),type(predicted))
ytest = pd.Series(y_test).values
print((ytest.shape[0]))

<class 'pandas.core.series.Series'> <class 'numpy.ndarray'>
3855


In [8]:
#find the index where both are 1.
count = 0
fault = 0
predictedfault = 0
for i in range(predicted.shape[0]):
    if((predicted[i] == 1) and (ytest[i] == 1)):
        count += 1
    if(ytest[i] == 1):
        fault += 1
    if(predicted[i]==1):
        predictedfault += 1
print("No of fraud cases:",fault)
print("No of fauld cases predicted by model:",predictedfault)
print("No of fraud correctly predicted  as fault:",count)

No of fraud cases: 246
No of fauld cases predicted by model: 10
No of fraud correctly predicted  as fault: 7


In [9]:
#with Smote
sm = RandomOverSampler(random_state=42)
features,labels = sm.fit_sample(car_features,car_label)
print(features.shape,labels.shape)
X_train,X_test,y_train,y_test = train_test_split(features,labels,random_state=3,test_size=0.25)


(28992, 52) (28992,)


In [10]:
#model object
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)
print('Random forest classifier:')
predicted = model.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,model.predict(X_test)) * 100,2))


Random forest classifier:
<class 'numpy.ndarray'> (7248,) [1 1 0 ..., 0 0 1]
Accuracy is  99.5


In [11]:
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

Confusion Matrix:
 [[3587   36]
 [   0 3625]]
Accuracy: 99.5033112583
Sensitivity: 100.0
Specificity: 99.0063483301


In [12]:
#converting pandas.core.series.Series to numpy.ndarray
ytest = pd.Series(y_test).values
print((ytest.shape[0]))
print(predicted.shape[0])
print(ytest)
print()

7248
7248
[1 1 0 ..., 0 0 1]



In [13]:
#find the index where both are 1.
count = 0
fault = 0
predictedfault = 0
for i in range(predicted.shape[0]):
    
    if((predicted[i] == 1) and (ytest[i] == 1)):
        count += 1
    if(ytest[i] == 1):
        fault += 1
    if(predicted[i]==1):
        predictedfault += 1
print("No of fraud cases:",fault)
print("No of fauld cases predicted by model:",predictedfault)
print("No of fraud correctly predicted  as fault:",count)

No of fraud cases: 3625
No of fauld cases predicted by model: 3661
No of fraud correctly predicted  as fault: 3625


In [14]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_test, predicted, average='micro')
precision_recall_fscore_support(y_test, predicted, average='macro')
precision_recall_fscore_support(y_test, predicted, average='weighted')


(0.99508195386850107, 0.99503311258278149, 0.99503298323887246, None)

In [15]:
#cross-validation 10-FOLD cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score



print('Random forest classifier:')

sm = RandomOverSampler(random_state=100)

features,labels = sm.fit_sample(car_features,car_label)

#convert np.array to dataframe 
df_features = pd.DataFrame(features)
df_lables = pd.DataFrame(labels)


Random forest classifier:


In [16]:
#add the both the labels and features
df_features['FraudFound']= df_lables

#shuffle the featrues
new_combine_features = df_features.set_index(np.random.permutation(df_features.index))


new_label = new_combine_features['FraudFound']

#drop the FraudFound attribute
new_combine_features.drop(['FraudFound'],inplace=True,axis=1)

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=10)
model=RandomForestClassifier(n_estimators=100) 

results = model_selection.cross_validate(estimator=model,X=new_combine_features,y=new_label,cv=kfold,scoring=scoring)

print(np.mean(results['test_accuracy']))
print(np.mean(results['test_precision']))
print(np.mean(results['test_recall']))
print(np.mean(results['test_f1_score']))



0.995964827348
0.943190953969
1.0
0.969702959954


In [ ]:
#test only on original datasets:

